In [1]:
from scipy.optimize import fsolve
import numpy as np

In [7]:
utility_params = {
                  'alpha' : 0.3,
                  'beta'  : 0.7
                    }

In [8]:
utility_params['alpha']

0.3

In [9]:
budget_params = {
                 'price1': 1,
                 'price2': 2,
                 'income': 30
    
                }

In [10]:
# functions for the first order conditions

In [11]:
def optimal_good1(quantity, utility_dict, budget_dict):
    
    alpha = utility_dict['alpha']
    beta  = utility_dict['beta']
    price1 = budget_dict['price1']
    
    qty1 = quantity[0]
    qty2 = quantity[1]
    lmbda = quantity[2]
    
    marginal_utility = alpha * qty1 ** (alpha - 1) * qty2** beta 
    marginal_cost    = lmbda * price1
    
    foc = marginal_utility - marginal_cost
    
    return foc

In [12]:
guess_sol = [2, 4, 0.5]

In [13]:
optimal_good1(guess_sol, utility_params, budget_params)

-0.01264856218625876

## challenge!

write out functions that return

1. distance from optimal qty for good 2
2. how much budget I have left over

In [14]:
def optimal_good2(quantity, utility_dict, budget_dict):
    
    alpha = utility_dict['alpha']
    beta  = utility_dict['beta']
    price2 = budget_dict['price2']
    
    qty1 = quantity[0]
    qty2 = quantity[1]
    lmbda = quantity[2]
    
    marginal_utility = beta * qty1 ** (alpha) * qty2**(beta - 1) 
    marginal_cost    = lmbda * price2
    
    foc = marginal_utility - marginal_cost
    
    return foc

In [27]:
def budget_constraint(quantity, budget_dict):
    
    price1 = budget_dict['price1']
    price2 = budget_dict['price2']
    income = budget_dict['income']
    
    qty1, qty2, _ = quantity
    
    leftover = income -  price1 * qty1 - price2 * qty2
    
    return leftover

In [28]:
def crit(quantity, utility_dict, budget_dict):
    
    foc_vector = np.empty_like(quantity)
    
    foc_vector[0] = optimal_good1(quantity, utility_dict, budget_dict)
    foc_vector[1] = optimal_good2(quantity, utility_dict, budget_dict)
    foc_vector[2] = budget_constraint(quantity, budget_dict)
    
    return foc_vector

In [29]:
crit(guess_sol, utility_params, budget_params)

array([-1.26485622e-02, -4.31423323e-01,  2.00000000e+01])

## solve! 

In [30]:
params = (utility_params, budget_params)

In [31]:
start_vals = np.array([1.0, 1.0, 1.0])

In [52]:
my_sol, infodict, ier, _ = fsolve(crit, start_vals, args = params, full_output= True)

In [49]:
my_sol

array([ 9.        , 10.5       ,  0.33418273])

In [53]:
infodict['fvec']

array([ 1.66533454e-16,  0.00000000e+00, -3.55271368e-15])

In [54]:
ier

1

In [33]:
help(fsolve)

Help on function fsolve in module scipy.optimize.minpack:

fsolve(func, x0, args=(), fprime=None, full_output=0, col_deriv=0, xtol=1.49012e-08, maxfev=0, band=None, epsfcn=None, factor=100, diag=None)
    Find the roots of a function.
    
    Return the roots of the (non-linear) equations defined by
    ``func(x) = 0`` given a starting estimate.
    
    Parameters
    ----------
    func : callable ``f(x, *args)``
        A function that takes at least one (possibly vector) argument,
        and returns a value of the same length.
    x0 : ndarray
        The starting estimate for the roots of ``func(x) = 0``.
    args : tuple, optional
        Any extra arguments to `func`.
    fprime : callable ``f(x, *args)``, optional
        A function to compute the Jacobian of `func` with derivatives
        across the rows. By default, the Jacobian will be estimated.
    full_output : bool, optional
        If True, return optional outputs.
    col_deriv : bool, optional
        Specify wheth

## optimize

In [55]:
from scipy import optimize

In [80]:
np.random.seed(42)

In [57]:
data_properties = {
                    'n_obs': 10000,
                    'beta' : np.array([2, -0.25])
                    }

In [65]:
data_properties['beta'].shape

(2,)

In [84]:
def gen_data(data_properties):
    
    n_obs = data_properties['n_obs']
    beta  = data_properties['beta']
    
    # make explanatory variables
    iota  = np.ones(n_obs).reshape(n_obs, 1)
    price = np.random.uniform(size = [n_obs, 1])
    
    X = np.hstack([iota, price])
    epsilon = np.random.gumbel(size = [n_obs, 2])
    
    beta_tilde = np.hstack([
                            np.zeros(beta.shape[0]).reshape(beta.shape[0], 1),
                            beta.reshape(beta.shape[0], 1)
                            ]
                            )
    
    utility = X @ beta_tilde + epsilon
    
    choice  = np.argmax(utility, axis = 1)
    
    return choice, X

In [85]:
y, X = gen_data(data_properties)

In [92]:
from scipy.stats import logistic

In [93]:
def calc_choice_prob(expl_var, beta_guess):
    
    choice_pr = logistic.cdf(expl_var @ beta_guess)
    
    return choice_pr

In [94]:
def log_like_i(beta, y , X):
    
    choice_prob = calc_choice_prob(X, beta)
    
    ll_i = np.log( (y == 1) * choice_prob + ( y== 0) * (1 - choice_prob) )
    
    return ll_i

In [95]:
def crit(beta, y, X):
    
    ll_i = log_like_i(beta, y, X)
    
    return -(ll_i.sum())

In [98]:
start_val = np.array([0.5, 0.5])

In [103]:
my_output = optimize.minimize( crit,
                              start_val,
                              args   = (y, X),
                              method = 'L-BFGS-B',
                              tol    = 1e-10
                            )

In [104]:
dir(my_output)

['fun', 'hess_inv', 'jac', 'message', 'nfev', 'nit', 'status', 'success', 'x']

In [105]:
beta_hat = my_output.x

In [106]:
beta_hat

array([ 2.03773249, -0.33563105])

In [107]:
my_output

      fun: 3931.8953697905886
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.54747351e-05, -4.54747351e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 27
      nit: 7
   status: 0
  success: True
        x: array([ 2.03773249, -0.33563105])

In [122]:
inv_hessian = my_output.hess_inv.todense()

In [125]:
my_std_error = np.sqrt(np.diag(inv_hessian))

In [126]:
my_std_error

array([0.06100342, 0.10390294])

In [119]:
from scipy import linalg as la

In [120]:
inv_hessian = la.pinv2(hessian)

In [121]:
inv_hessian

array([[1149.20960629,  590.59440867],
       [ 590.59440867,  396.1429174 ]])

In [96]:
help(optimize.minimize)

Help on function minimize in module scipy.optimize._minimize:

minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
    Minimization of scalar function of one or more variables.
    
    Parameters
    ----------
    fun : callable
        The objective function to be minimized.
    
            ``fun(x, *args) -> float``
    
        where x is an 1-D array with shape (n,) and `args`
        is a tuple of the fixed parameters needed to completely
        specify the function.
    x0 : ndarray, shape (n,)
        Initial guess. Array of real elements of size (n,),
        where 'n' is the number of independent variables.
    args : tuple, optional
        Extra arguments passed to the objective function and its
        derivatives (`fun`, `jac` and `hess` functions).
    method : str or callable, optional
        Type of solver.  Should be one of
    
            - 'Nelder-Mead' :ref:`(see here) <op